In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [3]:
# Загрузка данных
current_directory = os.path.dirname(os.getcwd())
train_path = os.path.join(current_directory, 'csv')
train_data = pd.read_csv(os.path.join(train_path, 'train1.csv'))
test_data = pd.read_csv(os.path.join(train_path, 'test1.csv'))
sample_submission = pd.read_csv(os.path.join(train_path, 'sample_submission.csv'))

In [4]:
# Целевая переменная
target = train_data["Transported"].astype(int)
train_data.drop("Transported", axis=1, inplace=True)

In [5]:
# Функция для анализа данных
def analyze_data(df):
    print(f"Размер данных: {df.shape}")
    for col in df.columns:
        print(f"{col}: уникальных значений = {df[col].nunique()}, пропущенных значений = {df[col].isnull().sum()}")

In [6]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


In [7]:
analyze_data(train_data)

Размер данных: (8693, 13)
PassengerId: уникальных значений = 8693, пропущенных значений = 0
HomePlanet: уникальных значений = 3, пропущенных значений = 201
CryoSleep: уникальных значений = 2, пропущенных значений = 217
Cabin: уникальных значений = 6560, пропущенных значений = 199
Destination: уникальных значений = 3, пропущенных значений = 182
Age: уникальных значений = 80, пропущенных значений = 179
VIP: уникальных значений = 2, пропущенных значений = 203
RoomService: уникальных значений = 1273, пропущенных значений = 181
FoodCourt: уникальных значений = 1507, пропущенных значений = 183
ShoppingMall: уникальных значений = 1115, пропущенных значений = 208
Spa: уникальных значений = 1327, пропущенных значений = 183
VRDeck: уникальных значений = 1306, пропущенных значений = 188
Name: уникальных значений = 8473, пропущенных значений = 200


In [8]:
# Предобработка данных
def preprocessing(df):
    df = df.copy()


    df["Spent"] = df[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].sum(axis=1)


    if "PassengerId" in df.columns:
        group_pass = df["PassengerId"].str.split("_", expand=True).rename(columns={0: "GroupID", 1: "PassID"})
        group_pass["GroupID"] = group_pass["GroupID"].astype(int)
        df = df.join(group_pass).drop(["PassengerId", "PassID"], axis=1)

    # Разделение Cabin
    if "Cabin" in df.columns:
        cabin_split = df["Cabin"].str.split("/", expand=True).rename(columns={0: "CabinDeck", 1: "CabinNum", 2: "CabinSide"})
        df = df.join(cabin_split).drop("Cabin", axis=1)

    # Удаление ненужных колонок
    columns_to_drop = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "CabinNum", "Name", "Destination"]
    df = df.drop([col for col in columns_to_drop if col in df.columns], axis=1)

    # Добавление признака "NumRelatives" поиска числа родственников по каюте и общему имени
    if "GroupID" in df.columns:
        df["NumRelatives"] = df.groupby("GroupID")["GroupID"].transform("count") - 1

    return df

df_train_1 = preprocessing(train_data)
df_train_1.head()

,HomePlanet,CryoSleep,Age,VIP,Spent,GroupID,CabinDeck,CabinSide,NumRelatives
0,Europa,False,39.0,False,0.0,1,B,P,0
1,Earth,False,24.0,False,736.0,2,F,S,0
2,Europa,False,58.0,True,10383.0,3,A,S,1
3,Europa,False,33.0,False,5176.0,3,A,S,1
4,Earth,False,16.0,False,1091.0,4,F,S,0


In [9]:
analyze_data(df_train_1)

Размер данных: (8693, 9)
HomePlanet: уникальных значений = 3, пропущенных значений = 201
CryoSleep: уникальных значений = 2, пропущенных значений = 217
Age: уникальных значений = 80, пропущенных значений = 179
VIP: уникальных значений = 2, пропущенных значений = 203
Spent: уникальных значений = 2336, пропущенных значений = 0
GroupID: уникальных значений = 6217, пропущенных значений = 0
CabinDeck: уникальных значений = 8, пропущенных значений = 199
CabinSide: уникальных значений = 2, пропущенных значений = 199
NumRelatives: уникальных значений = 8, пропущенных значений = 0


In [12]:
# Отключаем предупреждение
pd.set_option('future.no_silent_downcasting', True)

# Функция passproc: обработка пропущенных данных
def passproc(df):
    df = df.copy()
    group_data = df.groupby("GroupID")

    # Заполнение пропусков
    for col in ["HomePlanet", "CryoSleep", "VIP", "CabinDeck", "CabinSide"]:
        if col in df.columns:
            df[col] = group_data[col].transform(lambda group: group.fillna(group.mode().iloc[0] if not group.mode().empty else df[col].mode().iloc[0]))

    for col in ["Age", "Spent"]:
        if col in df.columns:
            df[col] = group_data[col].transform(lambda group: group.fillna(group.mean() if not group.mean() is np.nan else df[col].mean()))

    return df

df_train_2 = passproc(df_train_1)

In [13]:
analyze_data(train_data)

Размер данных: (8693, 13)
PassengerId: уникальных значений = 8693, пропущенных значений = 0
HomePlanet: уникальных значений = 3, пропущенных значений = 201
CryoSleep: уникальных значений = 2, пропущенных значений = 217
Cabin: уникальных значений = 6560, пропущенных значений = 199
Destination: уникальных значений = 3, пропущенных значений = 182
Age: уникальных значений = 80, пропущенных значений = 179
VIP: уникальных значений = 2, пропущенных значений = 203
RoomService: уникальных значений = 1273, пропущенных значений = 181
FoodCourt: уникальных значений = 1507, пропущенных значений = 183
ShoppingMall: уникальных значений = 1115, пропущенных значений = 208
Spa: уникальных значений = 1327, пропущенных значений = 183
VRDeck: уникальных значений = 1306, пропущенных значений = 188
Name: уникальных значений = 8473, пропущенных значений = 200


In [14]:
def onehot_encoding(df):
    df = df.copy()

    # One-Hot Encoding
    if "HomePlanet" in df.columns:
        dummies = pd.get_dummies(df["HomePlanet"], prefix="HomePlanet", drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop("HomePlanet", axis=1)

    if "CabinDeck" in df.columns:
        dummies = pd.get_dummies(df["CabinDeck"], prefix="CabinDeck", drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop("CabinDeck", axis=1)

    if "CabinSide" in df.columns:
        dummies = pd.get_dummies(df["CabinSide"], prefix="CabinSide", drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop("CabinSide", axis=1)


    for col in ["CryoSleep", "VIP"]:
        if col in df.columns:
            df[col] = df[col].astype(int)


    binary_columns = [
        "HomePlanet_Earth", "HomePlanet_Europa", "HomePlanet_Mars",
        "CabinDeck_A", "CabinDeck_B", "CabinDeck_C", "CabinDeck_D", 
        "CabinDeck_E", "CabinDeck_F", "CabinDeck_G", "CabinDeck_T", 
        "CabinSide_P", "CabinSide_S"
    ]
    for col in binary_columns:
        if col in df.columns:
            df[col] = df[col].astype(int)

    return df

df_train_3 = onehot_encoding(df_train_2)
df_train_3

,CryoSleep,Age,VIP,Spent,GroupID,NumRelatives,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T,CabinSide_P,CabinSide_S
0,0,39.0,0,0.0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0
1,0,24.0,0,736.0,2,0,1,0,0,0,0,0,0,0,1,0,0,0,1
2,0,58.0,1,10383.0,3,1,0,1,0,1,0,0,0,0,0,0,0,0,1
3,0,33.0,0,5176.0,3,1,0,1,0,1,0,0,0,0,0,0,0,0,1
4,0,16.0,0,1091.0,4,0,1,0,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41.0,1,8536.0,9276,0,0,1,0,1,0,0,0,0,0,0,0,1,0
8689,1,18.0,0,0.0,9278,0,1,0,0,0,0,0,0,0,0,1,0,0,1
8690,0,26.0,0,1873.0,9279,0,1,0,0,0,0,0,0,0,0,1,0,0,1
8691,0,32.0,0,4637.0,9280,1,0,1,0,0,0,0,0,1,0,0,0,0,1


In [15]:
analyze_data(df_train_3)

Размер данных: (8693, 19)
CryoSleep: уникальных значений = 2, пропущенных значений = 0
Age: уникальных значений = 108, пропущенных значений = 0
VIP: уникальных значений = 2, пропущенных значений = 0
Spent: уникальных значений = 2336, пропущенных значений = 0
GroupID: уникальных значений = 6217, пропущенных значений = 0
NumRelatives: уникальных значений = 8, пропущенных значений = 0
HomePlanet_Earth: уникальных значений = 2, пропущенных значений = 0
HomePlanet_Europa: уникальных значений = 2, пропущенных значений = 0
HomePlanet_Mars: уникальных значений = 2, пропущенных значений = 0
CabinDeck_A: уникальных значений = 2, пропущенных значений = 0
CabinDeck_B: уникальных значений = 2, пропущенных значений = 0
CabinDeck_C: уникальных значений = 2, пропущенных значений = 0
CabinDeck_D: уникальных значений = 2, пропущенных значений = 0
CabinDeck_E: уникальных значений = 2, пропущенных значений = 0
CabinDeck_F: уникальных значений = 2, пропущенных значений = 0
CabinDeck_G: уникальных значений 

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train_3, target, test_size=0.2, random_state=42, stratify=target
)


In [17]:
# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [31]:
import warnings

# Подавляем предупреждение
warnings.filterwarnings("ignore", category=UserWarning)

# Обучение модели
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)

# Предсказание на тестовых данных
rf_pred = rf_model.predict(X_test)

# Оценка точности
rf_accuracy = accuracy_score(y_test, rf_pred)
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")

Random Forest Accuracy: 0.7280


In [24]:
# Инициализация модели XGBoost
xgb_model = xgb.XGBClassifier(random_state=42, n_estimators=100, eval_metric='logloss')

# Обучение модели
xgb_model.fit(X_train, y_train)

# Предсказание на тестовых данных
xgb_pred = xgb_model.predict(X_test)

# Оценка точности
xgb_accuracy = accuracy_score(y_test, xgb_pred)
print(f"XGBoost Accuracy: {xgb_accuracy:.4f}")

XGBoost Accuracy: 0.7550


In [32]:
# Предсказания от моделей
rf_predictions = rf_model.predict(df_train_3)
xgb_predictions = xgb_model.predict(df_train_3)

In [33]:
# Совпадение предсказаний
agreement = np.mean(rf_predictions == xgb_predictions)
print(f"Процент совпадения предсказаний Random Forest и XGBoost: {agreement * 100:.2f}%")

Процент совпадения предсказаний Random Forest и XGBoost: 54.34%
